In [2]:
import planetengine

from modelscripts import MS98X_systemscript as systemscript
from modelscripts import MS98X_observerscript as observerscript
#from modelscripts import noisydoublephase_initialscript as initialscript
from modelscripts import sinudoublephase_initialscript as initialscript

model = planetengine.frame.Frame(
    systemscript.build(
        res = 64,
        ratio = 1.,
        aspect = 1.,
        periodic = False,
        creep_sR = 1.,
        ),
    observerscript.build(),
    initialscript.build(
        freq = 1.,
        phase = 0.9,
        cont_centre = 1.,
        cont_width = 1.,
        ),
    outputPath = '../data',
    instanceID = 'annulusSwarmBug',
    )

In [3]:
testobj = initialscript.build()

In [7]:
sorted(testobj.selfdict.items())

[('apply',
  <function modelscripts.sinudoublephase_initialscript.build.<locals>.apply(system)>),
 ('cont_centre', 0.5),
 ('cont_thickness', 0.035),
 ('cont_width', 0.5),
 ('freq', 1.0),
 ('inputs',
  {'cont_centre': 0.5,
   'cont_thickness': 0.035,
   'cont_width': 0.5,
   'freq': 1.0,
   'pert': 0.2,
   'phase': 0.0}),
 ('pert', 0.2),
 ('phase', 0.0),
 ('script',
  '/home/jovyan/workspace/user_data/modelscripts/sinudoublephase_initialscript.py')]

In [9]:
varsOfState = model.system.varsOfState

In [13]:
varsOfState

[((('temperatureField',
    <underworld.mesh._meshvariable.MeshVariable at 0x7f3921fe76a0>),),
  ('mesh',
   <underworld.mesh._spherical_mesh.FeMesh_Annulus at 0x7f38f14e8d30>)),
 ((('materialVar',
    <underworld.swarm._swarmvariable.SwarmVariable at 0x7f393032d908>),),
  ('swarm', <underworld.swarm._swarm.Swarm at 0x7f3921f98438>))]

In [ ]:
varsOfState = [
    ((("temperatureField", temperatureField),), ("mesh", mesh)),
    ((("materialVar", materialVar),), ("swarm", swarm)),
    ]

In [ ]:
varsOfState = [(model.system.temperatureField, 'temperatureField', 'mesh'), (model.system.materialVar, 'materialVar', 'swarm')]

In [18]:
type(model.system.materialVar)

underworld.swarm._swarmvariable.SwarmVariable

In [16]:
import underworld as uw

In [17]:
type(model.system.temperatureField) == uw.mesh._meshvariable.MeshVariable

True

In [19]:
type(model.system.materialVar) == uw.swarm._swarmvariable.SwarmVariable

True

In [20]:
varsOfState = [
    (model.system.temperatureField, 'temperatureField', 'mesh'),
    (model.system.materialVar, 'materialVar', 'swarm'),
    ]

In [25]:
testdict = {'a': 1}

In [29]:
'b' in testdict

False

In [30]:
model.system.temperatureField.mesh

In [31]:
model.system.temperatureField.mesh == model.system.velocityField.mesh

True

In [ ]:
def varsOnDisk(varsOfState, directory, mode = 'save', blackhole = [0., 0.])
    substrates = []
    substrateNames = []
    substrateHandles = {}
    extension = '.h5'
    for var, varName in varsOfState:
        if type(var) == uw.mesh._meshvariable.MeshVariable:
            substrate = var.mesh
            substrateName = 'mesh'
        elif type(var) == uw.swarm._swarmvariable.SwarmVariable:
            substrate = var.swarm
            substrateName = 'swarm'
        else:
            raise Exception('Variable type not recognised.')
        if not substrate in substrates:
            if substrateName in substrateNames:
                nameFound = False
                suffix = 0
                while not nameFound:
                    adjustedSubstrateName = substrateName + '_' + str(suffix)
                    if not adjustedSubstrateName in substrateNames:
                        substrateName = adjustedSubstrateName
                        nameFound = True
                    else:
                        suffix += 1
            substrateNames.append(substrateName)
            if mode == 'save':
                handle = substrate.save(os.path.join(directory, substrateName + extension))
                substrateHandles[substrateName] = handle
            elif mode == 'load':
                if type(substrate) == uw.swarm.Swarm:
                    with substrate.deform_swarm():
                        substrate.particleCoordinates.data[:] = blackhole
                substrate.load(os.path.join(directory, substrateName + extension))
            else:
                raise Exception("Disk mode not recognised.")
            substrates.append(substrate)
        else:
            if mode == 'save':
                handle = substrateHandles[substrateNames]
        if mode == 'save':
            var.save(os.path.join(directory, varName + extension), handle)
        elif mode == 'load':
            var.load(os.path.join(directory, varName + extension), handle)
        else:
            raise Exception("Disk mode not recognised.")

In [ ]:
def initialFn(init)